
# Como começar com Deep Learning em PyTorch

Este notebook mostra os conceitos fundamentais de **Deep Learning** usando o PyTorch.  
Nosso foco será comparar uma **rede rasa** com uma **rede profunda**, destacando a diferença de representação.

> Dica: execute cada célula em sequência. Ao final, compare as métricas.


## 1) Importando bibliotecas

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu")

device



## 2) Preparando um dataset simples

Vamos criar um dataset artificial de classificação binária.  
Ele é pequeno e didático, adequado para comparar arquiteturas rasas e profundas.


In [ ]:
# Dataset sintético
X, y = make_classification(
    n_samples=2000, n_features=20, n_informative=10, n_classes=2, random_state=SEED
)

# Padronização
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Treino/validação
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=SEED, stratify=y
)

# Tensores
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
X_val   = torch.tensor(X_val,   dtype=torch.float32).to(device)
y_val   = torch.tensor(y_val,   dtype=torch.long).to(device)

X_train.shape, X_val.shape, y_train.shape, y_val.shape



## 3) Definindo modelos

### 3.1 Rede rasa (1 camada oculta)


In [ ]:
class ShallowMLP(nn.Module):
    def __init__(self, n_in, n_hidden, n_out):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_in, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_out)
        )
    def forward(self, x):
        return self.net(x)

shallow_model = ShallowMLP(20, 16, 2).to(device)
shallow_model



### 3.2 Rede profunda (3 camadas ocultas)


In [ ]:
class DeepMLP(nn.Module):
    def __init__(self, n_in, n_hidden, n_out):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_in, n_hidden), nn.ReLU(),
            nn.Linear(n_hidden, n_hidden), nn.ReLU(),
            nn.Linear(n_hidden, n_hidden), nn.ReLU(),
            nn.Linear(n_hidden, n_out)
        )
    def forward(self, x):
        return self.net(x)

deep_model = DeepMLP(20, 64, 2).to(device)
deep_model



## 4) Função de treino

O ciclo de treino segue três passos: **forward**, **cálculo da loss** e **backward + otimização**.
A métrica usada aqui será **acurácia de validação**.


In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, epochs=20, lr=1e-3):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    history = {"train_loss": [], "val_loss": [], "val_acc": []}

    for epoch in range(epochs):
        # Treino
        model.train()
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Validação
        model.eval()
        with torch.no_grad():
            val_pred = model(X_val)
            val_loss = loss_fn(val_pred, y_val).item()
            val_acc = (val_pred.argmax(1) == y_val).float().mean().item()

        history["train_loss"].append(loss.item())
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(f"Época {epoch+1:02d} | "
              f"Loss treino: {loss.item():.4f} | "
              f"Loss val: {val_loss:.4f} | "
              f"Acurácia val: {val_acc:.4f}")

    return history



## 5) Comparando desempenho

### 5.1 Treinando rede rasa


In [ ]:
hist_shallow = train_model(shallow_model, X_train, y_train, X_val, y_val, epochs=20, lr=1e-3)



### 5.2 Treinando rede profunda


In [ ]:
hist_deep = train_model(deep_model, X_train, y_train, X_val, y_val, epochs=20, lr=1e-3)



## 6) Conclusão

- A **rede rasa** consegue aprender parte dos padrões, mas tem **limitação de representação**.  
- A **rede profunda** tende a obter melhor desempenho, pois representa relações mais complexas.  
- Redes profundas exigem mais **dados**, **cálculo** e **regularização** para evitar overfitting.
